In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [2]:
gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]
tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
def preprocess_sentence(w):
    w = w.lower().strip()

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

    w = w.strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [4]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [5]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

    return tensor, lang_tokenizer

In [6]:
def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
    targ_lang, inp_lang = create_dataset(path, num_examples)

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [7]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [8]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=False,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
    

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [9]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

  def call(self, x, hidden):
    # enc_output shape == (batch_size, max_length, hidden_size)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x, initial_state=hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [10]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [16]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # storing the attention weights to plot later on
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [17]:
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

In [11]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [13]:
num_examples = 400000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset('rus.txt', num_examples)
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
max_length_targ,max_length_inp

(17, 21)

In [14]:
BUFFER_SIZE = len(input_tensor)+100
BATCH_SIZE = 1024
embedding_dim = 300
units = 512
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor, target_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)


In [15]:
EPOCHS = 50
loss_old = 100
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for batch,(inp, targ) in enumerate(dataset):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss


    current_loss = total_loss / batch
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      current_loss))
    if current_loss   > loss_old:
        print('loss начал расти, обучение остановленно')
        break
    loss_old =  current_loss

    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Loss 1.9431
Time taken for 1 epoch 246.34224367141724 sec

Epoch 2 Loss 1.5221
Time taken for 1 epoch 228.1304533481598 sec

Epoch 3 Loss 1.2592
Time taken for 1 epoch 227.98734307289124 sec

Epoch 4 Loss 0.8731
Time taken for 1 epoch 226.79672932624817 sec

Epoch 5 Loss 0.5911
Time taken for 1 epoch 226.83770275115967 sec

Epoch 6 Loss 0.4278
Time taken for 1 epoch 226.86330342292786 sec

Epoch 7 Loss 0.3300
Time taken for 1 epoch 226.94638013839722 sec

Epoch 8 Loss 0.2659
Time taken for 1 epoch 225.68364834785461 sec

Epoch 9 Loss 0.2208
Time taken for 1 epoch 222.6905176639557 sec

Epoch 10 Loss 0.1875
Time taken for 1 epoch 222.6374707221985 sec

Epoch 11 Loss 0.1620
Time taken for 1 epoch 222.81083393096924 sec

Epoch 12 Loss 0.1416
Time taken for 1 epoch 222.59242701530457 sec

Epoch 13 Loss 0.1253
Time taken for 1 epoch 224.78644609451294 sec

Epoch 14 Loss 0.1119
Time taken for 1 epoch 227.91164875030518 sec

Epoch 15 Loss 0.1006
Time taken for 1 epoch 227.858763694763

In [18]:
translate('Я хочу кушать.')
translate('Здесь хорошо.')
translate('Я не смогу поехать.')
translate(u'Я люблю, когда идет снег.')

Input: <start> я хочу кушать . <end>
Predicted translation: i want to eat . <end> 
Input: <start> здесь хорошо . <end>
Predicted translation: it is good here . <end> 
Input: <start> я не смогу поехать . <end>
Predicted translation: i can't go . <end> 
Input: <start> я люблю , когда идет снег . <end>
Predicted translation: i love it when it's snowing . <end> 


In [20]:
translate('Я хочу сходить в кино.')
translate('Ты пойдешь завтра в школу?.')
translate('Встречаемся после ужина в парке.')
translate('Ты выйдешь за меня замуж?')
translate('Сегодня прекрасная погода, а что завтра?')
translate('Мне очень нравиться этот фильм, вы смотрели его?')

Input: <start> я хочу сходить в кино . <end>
Predicted translation: i want to go to the movies . <end> 
Input: <start> ты пойдешь завтра в школу ? . <end>
Predicted translation: will you go to school tomorrow ? <end> 
Input: <start> встречаемся после ужина в парке . <end>
Predicted translation: we'll meet in australia after dinner . <end> 
Input: <start> ты выйдешь за меня замуж ? <end>
Predicted translation: will you marry me ? <end> 
Input: <start> сегодня прекрасная погода , а что завтра ? <end>
Predicted translation: is the weather day busy tomorrow ? <end> 
Input: <start> мне очень нравиться этот фильм , вы смотрели его ? <end>
Predicted translation: i'd like the movie who sent you here . <end> 
